In [1]:
import pandas as pd
import numpy as np
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [133]:
premier_league_season23_24 = pd.read_csv('E0.csv')

In [83]:
teams = premier_league_season23_24['HomeTeam'].unique()

In [147]:
def clean_seasonal_data(dataset):
    dataset.drop(['HTHG','HTAG','HTR','Referee','BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH','AvgD',	'AvgA',	'B365>2.5',	'B365<2.5',	'P>2.5',	'P<2.5',	'Max>2.5',	'Max<2.5',	'Avg>2.5',	'Avg<2.5',	'AHh',	'B365AHH',	'B365AHA',	'PAHH',	'PAHA',	'MaxAHH',	'MaxAHA',	'AvgAHH',	'AvgAHA',	'B365CH',	'B365CD',	'B365CA',	'BWCH',	'BWCD',	'BWCA',	'IWCH',	'IWCD',	'IWCA',	'PSCH',	'PSCD',	'PSCA',	'WHCH',	'WHCD',	'WHCA',	'VCCH',	'VCCD',	'VCCA',	'MaxCH',	'MaxCD',	'MaxCA',	'AvgCH',	'AvgCD',	'AvgCA',	'B365C>2.5',	'B365C<2.5',	'PC>2.5',	'PC<2.5',	'MaxC>2.5',	'MaxC<2.5',	'AvgC>2.5',	'AvgC<2.5',	'AHCh',	'B365CAHH',	'B365CAHA',	'PCAHH',	'PCAHA',	'MaxCAHH',	'MaxCAHA',	'AvgCAHH',	'AvgCAHA' ], inplace =True, axis = 1)
    dataset = pd.get_dummies(premier_league_season23_24, columns=['HomeTeam', 'AwayTeam'])
    dummy_columns = dataset.select_dtypes(bool).columns
    dataset[dummy_columns] = dataset[dummy_columns].astype(int)
    dataset['Date'] = pd.to_datetime(dataset['Date'],format = '%d/%m/%Y' )
    return dataset


def get_team_matches(team_name, dataset):
    team_games =  dataset[dataset[dataset.filter(like=team_name).columns].gt(0.5).any(axis=1)]
    matchday_array = np.arange(1, len(team_games) + 1)
    team_games['matchday'] = matchday_array
    return team_games

def get_team_home_matches(team_name, dataset):
    team_matches = get_team_matches(team_name, dataset)
    team_home_games =  team_matches[team_matches[team_matches.filter(like=f"HomeTeam_{team_name}").columns].gt(0.5).any(axis=1)]
    return team_home_games

def get_team_away_matches(team_name, dataset):
    team_matches = get_team_matches(team_name, dataset)
    team_away_games =  team_matches[team_matches[team_matches.filter(like=f"AwayTeam_{team_name}").columns].gt(0.5).any(axis=1)]
    return team_away_games

def last_n_matches_form(team_games, team_name, n):
    team_form = [0]
    last_points =[0]
    for index, row in team_games.iterrows():
        if (row['FTR'] == 'H' and row[f"HomeTeam_{team_name}"] == 1) or (row['FTR'] == 'A' and row[f"AwayTeam_{team_name}"] == 1):
            last_points.append(3)
        elif row['FTR'] == 'D':
            last_points.append(1)
        else:
            last_points.append(0)
        if len(last_points)>n:
            del last_points[0]
        team_form.append(np.sum(last_points))
    del team_form[-1]
    return team_form


def team_current_points_total(team_name, dataset, which = 'all'):
    team_points = [0]
    last_points =[0]
    if which =='all':
        team_games =  get_team_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if (row['FTR'] == 'H' and row[f"HomeTeam_{team_name}"] == 1) or (row['FTR'] == 'A' and row[f"AwayTeam_{team_name}"] == 1):
                last_points.append(3)
            elif row['FTR'] == 'D':
                last_points.append(1)
            else:
                last_points.append(0)
            team_points.append(np.sum(last_points))     
    elif which == 'home':
        team_games =  get_team_home_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row['FTR'] == 'H':
                last_points.append(3)
            elif row['FTR'] == 'D':
                last_points.append(1)
            else:
                last_points.append(0)
            team_points.append(np.sum(last_points))
    elif which == 'away':
        team_games =  get_team_away_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row['FTR'] == 'A':
                last_points.append(3)
            elif row['FTR'] == 'D':
                last_points.append(1)
            else:
                last_points.append(0)
            team_points.append(np.sum(last_points))
    del team_points[-1]
    return team_points


def get_team_form(team_name, dataset, n, which = 'all'):
    if which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)   
    elif which =='all':
        team_games =  get_team_matches(team_name, dataset) 
    form_n_matches = last_n_matches_form(team_games, team_name, n)
    return form_n_matches


def get_team_goals_total(team_name, dataset, which = 'all'):
    team_goals = [0]
    goals_against = [0]
    last_goals =[0]
    last_goals_against = [0]
    team_goals_per_game = [0]
    against_goals_per_game = [0]
    if which =='all':
        team_games =  get_team_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row[f"HomeTeam_{team_name}"] == 1 :
                last_goals.append(row['FTHG'])
                last_goals_against.append(row['FTAG'])
            elif row[f"AwayTeam_{team_name}"] == 1:
                last_goals.append(row['FTAG'])
                last_goals_against.append(row['FTHG'])
            team_goals.append(np.sum(last_goals))
            goals_against.append(np.sum(last_goals_against))
            team_goals_per_game.append(team_goals[-1]/len(team_goals))
            against_goals_per_game.append(goals_against[-1]/len(goals_against))
    elif which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_goals.append(row['FTHG'])
            last_goals_against.append(row['FTAG'])
            team_goals.append(np.sum(last_goals))
            goals_against.append(np.sum(last_goals_against))
            team_goals_per_game.append(team_goals[-1]/len(team_goals))
            against_goals_per_game.append(goals_against[-1]/len(goals_against))
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_goals.append(row['FTAG'])
            last_goals_against.append(row['FTHG'])
            team_goals.append(np.sum(last_goals))
            goals_against.append(np.sum(last_goals_against))
            team_goals_per_game.append(team_goals[-1]/len(team_goals))
            against_goals_per_game.append(goals_against[-1]/len(goals_against))
    del team_goals[-1]
    del goals_against[-1]
    del team_goals_per_game[-1]
    del against_goals_per_game[-1]
    return team_goals, goals_against, team_goals_per_game, against_goals_per_game


def last_n_matches_goals(team_games, team_name, n):
    team_goals = [0]
    goals_against = [0]
    last_goals =[0]
    last_goals_against = [0]
    
    for index, row in team_games.iterrows():
        if row[f"HomeTeam_{team_name}"] == 1:
            last_goals.append(row['FTHG'])
            last_goals_against.append(row['FTAG'])
        elif row[f"AwayTeam_{team_name}"] == 1:
            last_goals.append(row['FTAG'])
            last_goals_against.append(row['FTHG'])
        if len(last_goals)>n:
            del last_goals[0]
            del last_goals_against[0]
        team_goals.append(np.sum(last_goals))
        goals_against.append(np.sum(last_goals_against))
    del team_goals[-1]
    del goals_against[-1]
    return team_goals, goals_against

def get_team_goals_n_matches(team_name, dataset, n, which = 'all'):
    if which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)   
    elif which =='all':
        team_games =  get_team_matches(team_name, dataset) 
    team_goals_n_matches, goals_against_n_matches = last_n_matches_goals(team_games, team_name, n)
    return  team_goals_n_matches, goals_against_n_matches


def get_team_shots_per_game(team_name, dataset, which = 'all', on_target = False):
    team_shots = [0]
    shots_against = [0]
    last_shots =[0]
    last_shots_against = [0]
    team_shots_per_game = [0]
    against_shots_per_game = [0]
    if on_target == False:
        home_shots = 'HS'
        away_shots = 'AS'
    else:
        home_shots = 'HST'
        away_shots = 'AST'
    if which =='all':
        team_games =  get_team_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row[f"HomeTeam_{team_name}"] == 1 :
                last_shots.append(row[home_shots])
                last_shots_against.append(row[away_shots])
            elif row[f"AwayTeam_{team_name}"] == 1:
                last_shots.append(row[away_shots])
                last_shots_against.append(row[home_shots])
            team_shots.append(np.sum(last_shots))
            shots_against.append(np.sum(last_shots_against))
            team_shots_per_game.append(team_shots[-1]/len(team_shots))
            against_shots_per_game.append(shots_against[-1]/len(shots_against))
    elif which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_shots.append(row[home_shots])
            last_shots_against.append(row[away_shots])
            team_shots.append(np.sum(last_shots))
            shots_against.append(np.sum(last_shots_against))
            team_shots_per_game.append(team_shots[-1]/len(team_shots))
            against_shots_per_game.append(shots_against[-1]/len(shots_against))
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_shots.append(row[away_shots])
            last_shots_against.append(row[home_shots])
            team_shots.append(np.sum(last_shots))
            shots_against.append(np.sum(last_shots_against))
            team_shots_per_game.append(team_shots[-1]/len(team_shots))
            against_shots_per_game.append(shots_against[-1]/len(shots_against))
    del team_shots[-1]
    del shots_against[-1]
    del team_shots_per_game[-1]
    del against_shots_per_game[-1]
    return team_shots_per_game, against_shots_per_game


def last_n_matches_shots(team_games, team_name, n, on_target = False):
    team_shots = [0]
    shots_against = [0]
    last_shots =[0]
    last_shots_against = [0]
    if on_target == False:
        home_shots = 'HS'
        away_shots = 'AS'
    else:
        home_shots = 'HST'
        away_shots = 'AST'
    for index, row in team_games.iterrows():
        if row[f"HomeTeam_{team_name}"] == 1:
            last_shots.append(row[home_shots])
            last_shots_against.append(row[away_shots])
        elif row[f"AwayTeam_{team_name}"] == 1:
            last_shots.append(row[away_shots])
            last_shots_against.append(row[home_shots])
        if len(last_shots)>n:
            del last_shots[0]
            del last_shots_against[0]
        team_shots.append(np.sum(last_shots))
        shots_against.append(np.sum(last_shots_against))
    del team_shots[-1]
    del shots_against[-1]
    return team_shots, shots_against

def get_team_shots_n_matches(team_name, dataset, n, which = 'all', on_target = False):
    if which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)   
    elif which =='all':
        team_games =  get_team_matches(team_name, dataset) 
    team_shots_n_matches, shots_against_n_matches = last_n_matches_shots(team_games, team_name, n, on_target = on_target)
    return  team_shots_n_matches, shots_against_n_matches


def get_team_cards_per_game(team_name, dataset, which = 'all', yellow = True):
    team_cards = [0]
    cards_against = [0]
    last_cards =[0]
    last_cards_against = [0]
    team_cards_per_game = [0]
    against_cards_per_game = [0]
    if yellow == True:
        home_cards = 'HY'
        away_cards = 'AY'
    else:
        home_cards = 'HR'
        away_cards = 'AR'
    if which =='all':
        team_games =  get_team_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row[f"HomeTeam_{team_name}"] == 1 :
                last_cards.append(row[home_cards])
                last_cards_against.append(row[away_cards])
            elif row[f"AwayTeam_{team_name}"] == 1:
                last_cards.append(row[away_cards])
                last_cards_against.append(row[home_cards])
            team_cards.append(np.sum(last_cards))
            cards_against.append(np.sum(last_cards_against))
            team_cards_per_game.append(team_cards[-1]/len(team_cards))
            against_cards_per_game.append(cards_against[-1]/len(cards_against))
    elif which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_cards.append(row[home_cards])
            last_cards_against.append(row[away_cards])
            team_cards.append(np.sum(last_cards))
            cards_against.append(np.sum(last_cards_against))
            team_cards_per_game.append(team_cards[-1]/len(team_cards))
            against_cards_per_game.append(cards_against[-1]/len(cards_against))
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_cards.append(row[away_cards])
            last_cards_against.append(row[home_cards])
            team_cards.append(np.sum(last_cards))
            cards_against.append(np.sum(last_cards_against))
            team_cards_per_game.append(team_cards[-1]/len(team_cards))
            against_cards_per_game.append(cards_against[-1]/len(cards_against))
    del team_cards[-1]
    del cards_against[-1]
    del team_cards_per_game[-1]
    del against_cards_per_game[-1]
    return team_cards_per_game, against_cards_per_game

def get_team_xg_per_game(team_name, dataset, which = 'all'):
    team_shots = [0]
    shots_against = [0]
    last_shots =[0]
    last_shots_against = [0]
    team_shots_per_game = [0]
    against_shots_per_game = [0]
    home_shots = 'Home xG'
    away_shots = 'Away xG'
    if which =='all':
        team_games =  get_team_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row[f"HomeTeam_{team_name}"] == 1 :
                last_shots.append(row[home_shots])
                last_shots_against.append(row[away_shots])
            elif row[f"AwayTeam_{team_name}"] == 1:
                last_shots.append(row[away_shots])
                last_shots_against.append(row[home_shots])
            team_shots.append(np.sum(last_shots))
            shots_against.append(np.sum(last_shots_against))
            team_shots_per_game.append(team_shots[-1]/len(team_shots))
            against_shots_per_game.append(shots_against[-1]/len(shots_against))
    elif which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_shots.append(row[home_shots])
            last_shots_against.append(row[away_shots])
            team_shots.append(np.sum(last_shots))
            shots_against.append(np.sum(last_shots_against))
            team_shots_per_game.append(team_shots[-1]/len(team_shots))
            against_shots_per_game.append(shots_against[-1]/len(shots_against))
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_shots.append(row[away_shots])
            last_shots_against.append(row[home_shots])
            team_shots.append(np.sum(last_shots))
            shots_against.append(np.sum(last_shots_against))
            team_shots_per_game.append(team_shots[-1]/len(team_shots))
            against_shots_per_game.append(shots_against[-1]/len(shots_against))
    del team_shots[-1]
    del shots_against[-1]
    del team_shots_per_game[-1]
    del against_shots_per_game[-1]
    return team_shots_per_game, against_shots_per_game


def last_n_matches_xg(team_games, team_name, n):
    team_shots = [0]
    shots_against = [0]
    last_shots =[0]
    last_shots_against = [0]
    home_shots = 'Home xG'
    away_shots = 'Away xG'
    for index, row in team_games.iterrows():
        if row[f"HomeTeam_{team_name}"] == 1:
            last_shots.append(row[home_shots])
            last_shots_against.append(row[away_shots])
        elif row[f"AwayTeam_{team_name}"] == 1:
            last_shots.append(row[away_shots])
            last_shots_against.append(row[home_shots])
        if len(last_shots)>n:
            del last_shots[0]
            del last_shots_against[0]
        team_shots.append(np.sum(last_shots))
        shots_against.append(np.sum(last_shots_against))
    del team_shots[-1]
    del shots_against[-1]
    return team_shots, shots_against

def get_team_xg_n_matches(team_name, dataset, n, which = 'all'):
    if which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)   
    elif which =='all':
        team_games =  get_team_matches(team_name, dataset) 
    team_shots_n_matches, shots_against_n_matches = last_n_matches_shots(team_games, team_name, n)
    return  team_shots_n_matches, shots_against_n_matches



import requests
from bs4 import BeautifulSoup

# URL of the website to scrape
url = "https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures"

def scrape_data(url, season = '2023-2024'):
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Find the table with match data
        table = soup.find('table', {'id': f'sched_{season}_9_1'})
    
        # Initialize a list to hold match data
        matches = []
    
        # Iterate through the rows of the table
        for row in table.find_all('tr'):
            # Get all the columns in the row
            cols = row.find_all('td')
    
            # Check if the row contains match data
            if len(cols) > 0:
                # Extract relevant data from the columns
                date = cols[1].text.strip()
                home_team = cols[3].text.strip()
                home_xg = cols[4].text.strip()
                away_xg = cols[6].text.strip()
                away_team = cols[7].text.strip()
                if len(date)>5:
                #     score_link_tag = soup.find_all('td', {'data-stat': 'score'})[len(matches)].find('a')
                #     score_link = score_link_tag['href']
                #     base_url = "https://fbref.com"
                #     full_score_link = base_url + score_link
                #     response2 = requests.get(full_score_link)
                #     soup2 = BeautifulSoup(response.content, 'html.parser')
                #     all_tr = soup2.find_all('tr')
                #     for i in range(len(all_tr)):
                #         if 'Passing Accuracy' in str(all_tr[i]):
                #             pass_accuracy_row = all_tr[i+1]
                #         if 'Possession'in str(all_tr[i]):
                #             possession_row = all_tr[i+1]
                #     home_possession = int(possession_row.find_all('strong')[0].text[:-1])
                #     away_possession = int(possession_row.find_all('strong')[1].text[:-1])
                #     home_pass_accuracy = int(pass_accuracy_row.find_all('strong')[0].text[:-1])
                #     away_pass_accuracy = int(pass_accuracy_row.find_all('strong')[1].text[:-1])
                    matches.append((date, home_team, away_team, home_xg, away_xg))
                
                
    else:
        print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")
    xgoals = pd.DataFrame(matches, columns=['Date', 'Home Team', 'Away Team', 'Home xG', 'Away xG'])
    return xgoals

In [ ]:
xgoals_23_24 = scrape_data("https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures", season = '2023-2024')

In [137]:
xgoals_23_24 = xgoals_23_24[['Home Team', 'Away Team','Home xG', 'Away xG']]

In [111]:
teams = sorted(teams)
teams2 = sorted(xgoals_23_24['Home Team'].unique())

In [139]:
dict_teams = {}
for i in range(len(teams2)):
    dict_teams[teams2[i]] = teams[i]
xgoals_23_24 = xgoals_23_24.replace(dict_teams)    

In [141]:
premier_league_season23_24 = pd.merge(premier_league_season23_24, xgoals_23_24, left_on = ['HomeTeam', 'AwayTeam'], right_on = ['Home Team', 'Away Team'])

In [155]:
premier_league_season23_24 = clean_seasonal_data(premier_league_season23_24)

In [161]:
premier_league_season23_24['Home xG'] = pd.to_numeric(premier_league_season23_24['Home xG'], errors='coerce')
premier_league_season23_24['Away xG'] = pd.to_numeric(premier_league_season23_24['Away xG'], errors='coerce')

In [238]:
teams_datasets_23_24 = []
for i in range(len(teams)):
    team_name = teams[i]
    team_1_games =  get_team_matches(team_name, premier_league_season23_24)
    team_1_away_games = get_team_away_matches(team_name, premier_league_season23_24)
    team_1_home_games = get_team_home_matches(team_name, premier_league_season23_24)
    
    team_1_games['total_points'] = team_current_points_total(team_name, premier_league_season23_24)
    team_1_games['total_points_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_points'], np.nan)
    team_1_games['total_points_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_points'], np.nan)
    
    team_1_games['5_form'] = get_team_form(team_name, premier_league_season23_24, 5, 'all')
    team_1_games['5_form_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form'], np.nan)
    team_1_games['5_form_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form'], np.nan)
    
    team_1_games['10_form'] = get_team_form(team_name, premier_league_season23_24, 10, 'all')
    team_1_games['10_form_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form'], np.nan)
    team_1_games['10_form_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form'], np.nan)
    
    team_1_all_goals_stats = get_team_goals_total(team_name, premier_league_season23_24, 'all')
    team_1_home_goals_stats = get_team_goals_total(team_name, premier_league_season23_24, 'home')
    team_1_away_goals_stats = get_team_goals_total(team_name, premier_league_season23_24, 'away')
    
    team_1_games['total_goals'] = team_1_all_goals_stats[0]
    team_1_games['total_goals_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_goals'], np.nan)
    team_1_games['total_goals_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_goals'], np.nan)
    
    team_1_games['total_goals_against'] = team_1_all_goals_stats[1]
    team_1_games['total_goals_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_goals_against'], np.nan)
    team_1_games['total_goals_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_goals_against'], np.nan)
    
    team_1_games['total_goals_per_game'] = team_1_all_goals_stats[2]
    team_1_games['total_goals_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_goals_per_game'], np.nan)
    team_1_games['total_goals_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_goals_per_game'], np.nan)
    
    team_1_games['total_goals_against_per_game'] = team_1_all_goals_stats[3]
    team_1_games['total_goals_against_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_goals_against_per_game'], np.nan)
    team_1_games['total_goals_against_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_goals_against_per_game'], np.nan)
    
    team_1_games['5_form_goals_scored'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 5, 'all')[0]
    team_1_games['5_form_goals_scored_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_goals_scored'], np.nan)
    team_1_games['5_form_goals_scored_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_goals_scored'], np.nan)
    
    
    team_1_games['5_form_goals_against'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 5, 'all')[1]
    team_1_games['5_form_goals_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_goals_against'], np.nan)
    team_1_games['5_form_goals_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_goals_against'], np.nan)
    
    team_1_games['10_form_goals_scored'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 10, 'all')[0]
    team_1_games['10_form_goals_scored_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form_goals_scored'], np.nan)
    team_1_games['10_form_goals_scored_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_goals_scored'], np.nan)
    
    team_1_games['10_form_goals_against'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 10, 'all')[1]
    team_1_games['10_form_goals_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form_goals_against'], np.nan)
    team_1_games['10_form_goals_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_goals_against'], np.nan)
    
    
    team_1_games['total_shots_per_game'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'all')[0]
    team_1_games['total_shots_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_shots_per_game'], np.nan)
    team_1_games['total_shots_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_shots_per_game'], np.nan)
    
    team_1_games['total_shots_against_per_game'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'all')[1]
    team_1_games['total_shots_against_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_shots_against_per_game'], np.nan)
    team_1_games['total_shots_against_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_shots_against_per_game'], np.nan)
    
    team_1_games['total_shots_on_target_per_game'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'all', on_target = True)[0]
    team_1_games['total_shots_on_target_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_shots_on_target_per_game'], np.nan)
    team_1_games['total_shots_on_target_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_shots_on_target_per_game'], np.nan)
    
    team_1_games['total_shots_on_target_against_per_game'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'all', on_target = True)[1]
    team_1_games['total_shots_on_target_against_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_shots_on_target_against_per_game'], np.nan)
    team_1_games['total_shots_on_target_against_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_shots_on_target_against_per_game'], np.nan)
    
    
    team_1_games['5_form_shots'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'all', on_target = False)[0]
    team_1_games['5_form_shots_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_shots'], np.nan)
    team_1_games['5_form_shots_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_shots'], np.nan)
    
    
    team_1_games['5_form_shots_against'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'all', on_target = False)[1]
    team_1_games['5_form_shots_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_shots_against'], np.nan)
    team_1_games['5_form_shots_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_shots_against'], np.nan)
    
    team_1_games['10_form_shots'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'all', on_target = False)[0]
    team_1_games['10_form_shots_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_shots_against'], np.nan)
    team_1_games['10_form_shots_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_shots'], np.nan)
    
    team_1_games['10_form_shots_against'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'all', on_target = False)[1]
    team_1_games['10_form_shots_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form_shots_against'], np.nan)
    team_1_games['10_form_shots_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_shots_against'], np.nan)
    
    team_1_games['5_form_shots_on_target'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'all', on_target = True)[0]
    team_1_games['5_form_shots_on_target_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_shots_on_target'], np.nan)
    team_1_games['5_form_shots_on_target_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_shots_on_target'], np.nan)
    
    team_1_games['5_form_shots_on_target_against'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'all', on_target = True)[1]
    team_1_games['5_form_shots_on_target_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_shots_on_target_against'], np.nan)
    team_1_games['5_form_shots_on_target_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_shots_on_target_against'], np.nan)
    
    team_1_games['10_form_shots_on_target'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'all', on_target = True)[0]
    team_1_games['10_form_shots_on_target_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form_shots_on_target'], np.nan)
    team_1_games['10_form_shots_on_target_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_shots_on_target'], np.nan)
    
    
    team_1_games['10_form_shots_on_target_against'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'all', on_target = True)[1]
    team_1_games['10_form_shots_on_target_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form_shots_on_target_against'], np.nan)
    team_1_games['10_form_shots_on_target_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_shots_on_target_against'], np.nan)
    
    
    team_1_games['total_yellow_cards_per_game'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'all', yellow = True)[0]
    team_1_games['total_yellow_cards_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_yellow_cards_per_game'], np.nan)
    team_1_games['total_yellow_cards_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_yellow_cards_per_game'], np.nan)
    
    
    team_1_games['total_yellow_cards_against_per_game'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'all', yellow = True)[1]
    team_1_games['total_yellow_cards_against_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_yellow_cards_against_per_game'], np.nan)
    team_1_games['total_yellow_cards_against_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_yellow_cards_against_per_game'], np.nan)
    
    
    team_1_games['total_red_cards_per_game'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'all', yellow = False)[0]
    team_1_games['total_red_cards_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_red_cards_per_game'], np.nan)
    team_1_games['total_red_cards_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_red_cards_per_game'], np.nan)
    
    team_1_games['total_red_cards_against_per_game'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'all', yellow = False)[1]
    team_1_games['total_red_cards_against_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_red_cards_against_per_game'], np.nan)
    team_1_games['total_red_cards_against_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_red_cards_against_per_game'], np.nan)
    
    
    team_1_games['total_xg_per_game'] = get_team_xg_per_game(team_name, premier_league_season23_24, 'all')[0]
    team_1_games['total_xg_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_xg_per_game'], np.nan)
    team_1_games['total_xg_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_xg_per_game'], np.nan)
    
    team_1_games['total_xg_against_per_game'] = get_team_xg_per_game(team_name, premier_league_season23_24, 'all')[1]
    team_1_games['total_xg_against_per_game_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['total_xg_against_per_game'], np.nan)
    team_1_games['total_xg_against_per_game_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['total_xg_against_per_game'], np.nan)
    
    team_1_games['5_form_xg'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 5, 'all')[0]
    team_1_games['5_form_xg_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_xg'], np.nan)
    team_1_games['5_form_xg_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_xg'], np.nan)
    
    team_1_games['5_form_xg_against'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 5, 'all')[1]
    team_1_games['5_form_xg_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['5_form_xg_against'], np.nan)
    team_1_games['5_form_xg_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['5_form_xg_against'], np.nan)
    
    
    team_1_games['10_form_xg'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 10, 'all')[0]
    team_1_games['10_form_xg_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form_xg'], np.nan)
    team_1_games['10_form_xg_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_xg'], np.nan)
    
    team_1_games['10_form_xg_against'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 10, 'all')[1]
    team_1_games['10_form_xg_against_home_team'] = np.where(team_1_games[f"HomeTeam_{team_name}"] == 1, team_1_games['10_form_xg_against'], np.nan)
    team_1_games['10_form_xg_against_away_team'] = np.where(team_1_games[f"AwayTeam_{team_name}"] == 1, team_1_games['10_form_xg_against'], np.nan)
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_points_away'] = team_current_points_total(team_name, premier_league_season23_24, 'away')
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_points_home'] = team_current_points_total(team_name, premier_league_season23_24, 'home')
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_away'] = get_team_form(team_name, premier_league_season23_24, 5, 'away')
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_away'] = get_team_form(team_name, premier_league_season23_24, 10, 'away')
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_home'] = get_team_form(team_name, premier_league_season23_24, 5, 'home')
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_home'] = get_team_form(team_name, premier_league_season23_24, 10, 'home')
    
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_goals_away'] = team_1_away_goals_stats[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_goals_against_away'] = team_1_away_goals_stats[1]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_goals_per_game_away'] = team_1_away_goals_stats[2]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_goals_against_per_game_away'] = team_1_away_goals_stats[3]
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_goals_home'] = team_1_home_goals_stats[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_goals_against_home'] = team_1_home_goals_stats[1]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_goals_per_game_home'] = team_1_home_goals_stats[2]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_goals_against_per_game_home'] = team_1_home_goals_stats[3]
    
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_goals_scored_away'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 5, 'away')[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_goals_scored_away'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 10, 'away')[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_goals_against_away'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 5, 'away')[1]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_goals_against_away'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 10, 'away')[1]
    
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_goals_scored_home'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 5, 'home')[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_goals_scored_home'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 10, 'home')[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_goals_against_home'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 5, 'home')[1]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_goals_against_home'] = get_team_goals_n_matches(team_name, premier_league_season23_24, 10, 'home')[1]
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_shots_per_game_home'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'home')[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_shots_against_per_game_home'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'home')[1]
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_shots_per_game_away'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'away')[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_shots_against_per_game_away'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'away')[1]
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_shots_on_target_per_game_home'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'home', on_target = True)[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_shots_on_target_against_per_game_home'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'home', on_target = True)[1]
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_shots_on_target_per_game_away'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'away', on_target = True)[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_shots_on_target_against_per_game_away'] = get_team_shots_per_game(team_name, premier_league_season23_24, 'away', on_target = True)[1]
    
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_shots_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'away', on_target = False)[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_shots_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'away', on_target = False)[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_shots_against_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'away', on_target = False)[1]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_shots_against_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'away', on_target = False)[1]
    
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_shots_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'home', on_target = False)[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_shots_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'home', on_target = False)[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_shots_against_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'home', on_target = False)[1]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_shots_against_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'home', on_target = False)[1]
    
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_shots_on_target_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'away', on_target = True)[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_shots_on_target_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'away', on_target = True)[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_shots_on_target_against_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'away', on_target = True)[1]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_shots_on_target_against_away'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'away', on_target = True)[1]
    
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_shots_on_target_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'home', on_target = True)[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_shots_on_target_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'home', on_target = True)[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_shots_on_target_against_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 5, 'home', on_target = True)[1]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_shots_on_target_against_home'] = get_team_shots_n_matches(team_name, premier_league_season23_24, 10, 'home', on_target = True)[1]
    
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_yellow_cards_per_game_home'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'home', yellow = True)[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_yellow_cards_against_per_game_home'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'home', yellow = True)[1]
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_yellow_cards_per_game_away'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'away', yellow = True)[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_yellow_cards_against_per_game_away'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'away', yellow = True)[1]
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_red_cards_per_game_home'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'home', yellow = False)[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_red_cards_against_per_game_home'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'home', yellow = False)[1]
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_red_cards_per_game_away'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'away', yellow = False)[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_red_cards_against_per_game_away'] = get_team_cards_per_game(team_name, premier_league_season23_24, 'away', yellow = False)[1]
    
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_xg_per_game_home'] = get_team_xg_per_game(team_name, premier_league_season23_24, 'home')[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, 'total_xg_against_per_game_home'] = get_team_xg_per_game(team_name, premier_league_season23_24, 'home')[1]
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_xg_per_game_away'] = get_team_xg_per_game(team_name, premier_league_season23_24, 'away')[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, 'total_xg_against_per_game_away'] = get_team_xg_per_game(team_name, premier_league_season23_24, 'away')[1]
    
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_xg_away'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 5, 'away')[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_xg_away'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 10, 'away')[0]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '5_form_xg_against_away'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 5, 'away')[1]
    team_1_games.loc[team_1_games[f'AwayTeam_{team_name}'] == 1, '10_form_xg_against_away'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 10, 'away')[1]
    
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_xg_home'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 5, 'home')[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_xg_home'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 10, 'home')[0]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '5_form_xg_against_home'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 5, 'home')[1]
    team_1_games.loc[team_1_games[f'HomeTeam_{team_name}'] == 1, '10_form_xg_against_home'] = get_team_xg_n_matches(team_name, premier_league_season23_24, 10, 'home')[1]
    teams_datasets_23_24.append(team_1_games)


In [239]:
merged_teams_datasets_23_24 = teams_datasets_23_24[0]
for df in teams_datasets_23_24[1:]:
    merged_teams_datasets_23_24 = merged_teams_datasets_23_24.combine_first(df)

In [240]:
merged_teams_datasets_23_24.isna().sum().sum()

0

In [260]:
merged_teams_datasets_23_24

,Div,Date,Time,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Home Team,Away Team,Home xG,Away xG,Home_xg,Away_xg,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Chelsea,HomeTeam_Crystal Palace,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Newcastle,HomeTeam_Nott'm Forest,HomeTeam_Sheffield United,HomeTeam_Tottenham,HomeTeam_West Ham,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Bournemouth,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Chelsea,AwayTeam_Crystal Palace,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Newcastle,AwayTeam_Nott'm Forest,AwayTeam_Sheffield United,AwayTeam_Tottenham,AwayTeam_West Ham,AwayTeam_Wolves,matchday,total_points,total_points_home_team,total_points_away_team,5_form,5_form_home_team,5_form_away_team,10_form,10_form_home_team,10_form_away_team,total_goals,total_goals_home_team,total_goals_away_team,total_goals_against,total_goals_against_home_team,total_goals_against_away_team,total_goals_per_game,total_goals_per_game_home_team,total_goals_per_game_away_team,total_goals_against_per_game,total_goals_against_per_game_home_team,total_goals_against_per_game_away_team,5_form_goals_scored,5_form_goals_scored_home_team,5_form_goals_scored_away_team,5_form_goals_against,5_form_goals_against_home_team,5_form_goals_against_away_team,10_form_goals_scored,10_form_goals_scored_home_team,10_form_goals_scored_away_team,10_form_goals_against,10_form_goals_against_home_team,10_form_goals_against_away_team,total_shots_per_game,total_shots_per_game_home_team,total_shots_per_game_away_team,total_shots_against_per_game,total_shots_against_per_game_home_team,total_shots_against_per_game_away_team,total_shots_on_target_per_game,total_shots_on_target_per_game_home_team,total_shots_on_target_per_game_away_team,total_shots_on_target_against_per_game,total_shots_on_target_against_per_game_home_team,total_shots_on_target_against_per_game_away_team,5_form_shots,5_form_shots_home_team,5_form_shots_away_team,5_form_shots_against,5_form_shots_against_home_team,5_form_shots_against_away_team,10_form_shots,10_form_shots_home_team,10_form_shots_away_team,10_form_shots_against,10_form_shots_against_home_team,10_form_shots_against_away_team,5_form_shots_on_target,5_form_shots_on_target_home_team,5_form_shots_on_target_away_team,5_form_shots_on_target_against,5_form_shots_on_target_against_home_team,5_form_shots_on_target_against_away_team,10_form_shots_on_target,10_form_shots_on_target_home_team,10_form_shots_on_target_away_team,10_form_shots_on_target_against,10_form_shots_on_target_against_home_team,10_form_shots_on_target_against_away_team,total_yellow_cards_per_game,total_yellow_cards_per_game_home_team,total_yellow_cards_per_game_away_team,total_yellow_cards_against_per_game,total_yellow_cards_against_per_game_home_team,total_yellow_cards_against_per_game_away_team,total_red_cards_per_game,total_red_cards_per_game_home_team,total_red_cards_per_game_away_team,total_red_cards_against_per_game,total_red_cards_against_per_game_home_team,total_red_cards_against_per_game_away_team,total_xg_per_game,total_xg_per_game_home_team,total_xg_per_game_away_team,total_xg_against_per_game,total_xg_against_per_game_home_team,total_xg_against_per_game_away_team,5_form_xg,5_form_xg_home_team,5_form_xg_away_team,5_form_xg_against,5_form_xg_against_home_team,5_form_xg_against_away_team,10_form_xg,10_form_xg_home_team,10_form_xg_away_team,10_form_xg_against,10_form_xg_against_home_team,10_form_xg_against_away_team,total_points_away,total_points_home,5_form_away,10_form_away,5_form_home,10_form_home,total_goals_away,total_goals_against_away,total_goals_per_game_away,total_goals_against_per_game_away,total_goals_home,total_goals_against_home,total_goals_per_game_home,total_goals_against_per_

In [61]:
premier_league_season23_24

,Div,Date,Time,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Chelsea,HomeTeam_Crystal Palace,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Newcastle,HomeTeam_Nott'm Forest,HomeTeam_Sheffield United,HomeTeam_Tottenham,HomeTeam_West Ham,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Bournemouth,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Chelsea,AwayTeam_Crystal Palace,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Newcastle,AwayTeam_Nott'm Forest,AwayTeam_Sheffield United,AwayTeam_Tottenham,AwayTeam_West Ham,AwayTeam_Wolves,Home_xg,Away_xg
0,E0,2023-08-11,20:00,0,3,A,6,17,1,8,11,8,6,5,0,0,1,0,8.00,5.50,1.33,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.3,1.9
1,E0,2023-08-12,12:30,2,1,H,15,6,7,2,12,12,8,3,2,2,0,0,1.18,7.00,15.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.8,1.2
2,E0,2023-08-12,15:00,1,1,D,14,16,5,3,9,14,10,4,1,4,0,0,2.70,3.40,2.55,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.7,1.5
3,E0,2023-08-12,15:00,4,1,H,27,9,12,3,11,12,6,7,2,2,0,0,1.33,5.50,9.00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.5,1.9
4,E0,2023-08-12,15:00,0,1,A,19,9,9,2,12,6,10,4,0,2,0,0,2.20,3.40,3.30,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4.0,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2024-05-19,16:00,5,0,H,15,8,9,2,10,8,2,4,1,4,0,0,1.85,4.33,3.60,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.1,3.4
376,E0,2024-05-19,16:00,2,0,H,36,4,14,3,14,11,10,2,1,1,0,1,1.17,8.00,15.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.1,2.2
377,E0,2024-05-19,16:00,2,4,A,15,15,6,7,15,20,4,4,5,4,0,0,2.90,3.90,2.20,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2.5,0.9
378,E0,2024-05-19,16:00,3,1,H,28,3,12,2,3,12,11,2,0,1,0,0,1.08,12.00,21.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4.5,0.5


# co dodać

Target ---> wyliczenie prawdopodobieństwa każdego z 1x2 (opcjonalnie porówanie strzelonych bramek) i porównanie z kursasmi/prawdziwymi wynikami?

# oddsy na prawdopodobieństwo
# wartość lineupów /suma wages zawodników?
# średnia zdobywanych/traconych bramek w sezonie(poprzednich)
# pozycja w tabeli >>>>>>>> to samo co punkty
# podania total/dom/wyjazd średnia z ost 5/srednia sezonowa 
# posiadanie piłki dom/wyjazd
# forma dom/wyjazd -- GOTOWE
# brasmki drużyny i przeciwko -- GOTOWE
# xg
# strzały ------ ---
# kartki ----------
# h2h drużyny powyższe 
# punkty i forma ostatnich 5 praeciwników --- 